# test

In [ ]:
!nvidia-smi

In [ ]:
!nvcc -V

In [ ]:
!pgc++ -V

# Normal
[normal_C.cpp](/edit/normal_C.cpp)

In [ ]:
!nvcc --run normal_C.cpp -O2 -o normal_C

In [ ]:
!./normal_C 128

Try
Changing -O3 to -O0
Changing input paramater `./normal_C 128`. Higher than default, 1024, will take longer.

# OpenMP
[openmp.cpp](/edit/openmp.cpp)

In [ ]:
!nvcc --run -ccbin pgc++ -O2 -Xcompiler "-V19.4 -mp" openmp.cpp -o openmp

Try
Removing -mp from build command
Changing omp_set_num_threads( 6 ) to 2, 4, or more.
Changing input paramater `./normal_C 128`. Higher than default, 1024, will take longer.

# OpenACC
[openacc.cpp](/edit/openacc.cpp)

In [1]:
!nvcc --run -ccbin pgc++ -O2 -Xcompiler "-V19.4 -Bstatic_pgi -acc -ta=tesla:nordc" openacc.cpp -o openacc

No input given.
Running with N = 1024

Running Normal C: 1302.21 ms
Running OpenACC: Test passed.


No time??

In [2]:
!nvcc --run -ccbin pgc++ -O2 -Xcompiler "-V19.4 -Bstatic_pgi -acc -ta=tesla:nordc -ta=time" openacc.cpp -o openacc

No input given.
Running with N = 1024

Running Normal C: 1271.48 ms
Running OpenACC: CUPTI ERROR: cuptiActivityEnable(CUPTI_ACTIVITY_KIND_KERNEL) returned: CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, 
	 at ../src-cupti/prof_cuda_cupti.c:265.
Test passed.

Accelerator Kernel Timing data
/home/belt/git/computeWorks_examples/computeWorks_mm/jupyter/openacc.cpp
  _Z7openACCifPKfS0_fPfRKi  NVIDIA  devicenum=0
    time(us): 10,410
    43: compute region reached 5 times
        49: kernel launched 5 times
            grid: [8x1024]  block: [128]
            elapsed time(us): total=11,760 max=3,107 min=2,085 avg=2,352
    43: data region reached 10 times
        43: data copyin transfers: 10
             device time(us): total=7,108 max=1,194 min=647 avg=710
        58: data copyout transfers: 5
             device time(us): total=3,302 max=712 min=644 avg=660


Why error? While if you are running the newest driver, 410.67, profiling requires elevated permissions

In [ ]:

!sudo LD_LIBRARY_PATH=/usr/local/cuda/extra/CUPTI/lib64:$LD_LIBRARY_PATH ./openacc

[sudo] password for belt: 

# BLAS
[blas.cpp](/edit/blas.cpp)

In [ ]:
!nvcc --run -ccbin pgc++ -O2 -Xlinker "-lblas" -Xcompiler "-V19.4" blas.cpp -o blas

In [ ]:
!./blas 128

# cuBLAS
[cublas.cpp](/edit/cublas.cpp)

In [ ]:
!nvcc --run -O2 -lcublas cublas.cpp -o cublas

In [ ]:
!./cublas 128

# CUDA
[cuda.cu](/edit/cuda.cu)

In [ ]:
!nvcc --run -O2 cuda.cu -o cuda

In [ ]:
!./cuda 128